# Step0: Install environment

In [3]:
import vapoursynth as vs
import havsfunc as haf
import mvsfunc as mvf
from vapoursynth import core
import awsmfunc as awsf

# Step1: Select BDMV Largest M2TS and its MLPS

In [ ]:
x265_filename=r"C:\Users\heber\Music\How.to.Train.Your.Dragon.2.2014.2160p.BluRay.x265.DTS-louco@PTHOME\How.to.Train.Your.Dragon.2.2014.2160p.BluRay.x265.DTS-louco@PTHOME.mkv"

# Step2: BDMV to MKV

# Step3: Transcode

In [1]:
import os, pyperclip, re, subprocess
from pymediainfo import MediaInfo
from datetime import datetime
from utils import *
crf=18.0
qcomp=0.7
aq_mode=3
aq_s=0.7
bitdepth=8
newline="\r\n"
remotecwd="~/compressor"
remotehost="amd"
PROCUCT_DEST_DIR=r"C:\Users\heber\Music"
MOVIE=r"C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.Atmos.TrueHD.7.1-louco@PTHome.mkv"

## get remux capture and crop black borders

In [5]:
get_captures("REMUX", MOVIE)

Capture REMUX


## H264: Test params

In [12]:
x264_stmt=f"x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} --bframes 11 --me umh  --subme 11 \
--merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 \
--vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 \
--min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv -"

args=x264_stmt.replace(" --","\n --").split("\n")[1:]
args=sorted(args)
# print("\n".join(["x264 参数："]+args))

remote_x264stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0|python {os. getcwd()}\\aio_buffer.py|\
ssh {remotehost} "python3 {remotecwd}/aio_buffer.py|zstdcat|{x264_stmt} 2>&1 | tee x264.log"{newline}'

print("==========\n远程执行编码 x264:")
cmd=f'set MOVIE={MOVIE}{newline}'
cmd+=f'set TEST=YES{newline}'
cmd+=remote_x264stmt
cmd+=f'scp {remotehost}:~/264.mkv {os. getcwd()}\\264.mkv{newline}'
# cmd+=f'scp {remotehost}:~/264.mkv D:\\BT\\samples\\264.mkv{newline}'
cmd+=f'pause{newline}'
pyperclip.copy(cmd)
print(cmd)

远程执行编码 x264:
set MOVIE=C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.Atmos.TrueHD.7.1-louco@PTHome.mkv
set TEST=YES
vspipe.exe --y4m "C:\Users\heber\Documents\Compressor\vs.py" - |zstd -0|python C:\Users\heber\Documents\Compressor\aio_buffer.py|ssh amd "python3 ~/compressor/aio_buffer.py|zstdcat|x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 --crf 18.0 --qcomp 0.7 --aq-mode 3 --aq-strength 0.7 --bframes 11 --me umh  --subme 11 --merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 --vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 --min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv - 2>&1 | tee x264.log"
scp amd:~/264.mkv C:\Users\heber\Documents\Compressor\264.mkv
pause



## H264&H265

In [13]:
x264_stmt=f"x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} --bframes 11 --me umh  --subme 11 \
--merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 \
--vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 \
--min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv -"

x265_stmt=f"x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower --crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} \
--aq-strength {aq_s} --rd 4 --psy-rd 2.0 --psy-rdoq 1.0 --rdoq-level 2 --deblock -1:-1 --cbqpoffs -1 --crqpoffs -3 --ctu 32 \
--qg-size 8 --no-sao --no-sao-non-deblock --selective-sao 0 --tu-intra-depth 4 --tu-inter-depth 4 --no-opt-qp-pps \
--no-opt-ref-list-length-pps --aud --repeat-headers --me 3 --subme 5 --merange 48 --b-intra --limit-tu 0 --no-rect \
--no-amp --no-open-gop --pools + --keyint 240 --min-keyint 1 --bframes 8 --max-merge 4 --ref 4 --weightb --rc-lookahead 72 \
--scenecut 40 --no-strong-intra-smoothing --vbv-bufsize 160000 --vbv-maxrate 160000 --input-depth 10 --output 265.mkv -"


remote_stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0 -T4|python {os. getcwd()}\\aio_buffer.py|\
ssh {remotehost} "python3 {remotecwd}/aio_buffer.py|zstdcat| tee \
>({x264_stmt} 2>&1 | tee x264.log) \
>({x265_stmt} 2>&1 | tee x265.log) \
>/dev/null{newline}'


print("==========\n远程执行编码 x264+x265:")
cmd=f'set MOVIE={MOVIE}{newline}'
cmd=f'set TEST=NO{newline}'
cmd+=remote_stmt
cmd+=f'scp {remotehost}:~/264.mkv {os. getcwd()}\\264.mkv{newline}'
cmd+=f'scp {remotehost}:~/265.mkv {os. getcwd()}\\265.mkv{newline}'

cmd+=f'ssh {remotehost} "cat x264.log|sed \\"s/.*\\r//g\\" " >{os. getcwd()}\\x264.log & '
cmd+=f'ssh {remotehost} "cat x265.log|sed \\"s/.*\\r//g\\" " >{os. getcwd()}\\x265.log{newline}'
cmd+=f'pause{newline}'
pyperclip.copy(cmd)
print(cmd)

远程执行编码 x264+x265:
set TEST=NO
vspipe.exe --y4m "C:\Users\heber\Documents\Compressor\vs.py" - |zstd -0 -T4|python C:\Users\heber\Documents\Compressor\aio_buffer.py|ssh amd "python3 ~/compressor/aio_buffer.py|zstdcat| tee >(x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 --crf 18.0 --qcomp 0.7 --aq-mode 3 --aq-strength 0.7 --bframes 11 --me umh  --subme 11 --merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 --vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 --min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv - 2>&1 | tee x264.log) >(x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower --crf 18.0 --qcomp 0.7 --aq-mode 3 --aq-strength 0.7 --rd 4 --psy-rd 2.0 --psy-rdoq 1.0 --rdoq-level 2 --deblock -1:-1 --cbqpoffs -1 --crqpoffs -3 --ctu 32 --qg-size 8 --no-sao --no-sao-non-deblock --selective-sao 0 --tu-intra-depth 4 --tu-inter-depth 4 --no-op

## H264 & H265 offline

In [5]:
x264_stmt=f"x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} --bframes 11 --me umh  --subme 11 \
--merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 \
--vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 \
--min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv -"

x265_stmt=f"x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower --crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} \
--aq-strength {aq_s} --rd 4 --psy-rd 2.0 --psy-rdoq 1.0 --rdoq-level 2 --deblock -1:-1 --cbqpoffs -1 --crqpoffs -3 --ctu 32 \
--qg-size 8 --no-sao --no-sao-non-deblock --selective-sao 0 --tu-intra-depth 4 --tu-inter-depth 4 --no-opt-qp-pps \
--no-opt-ref-list-length-pps --aud --repeat-headers --me 3 --subme 5 --merange 48 --b-intra --limit-tu 0 --no-rect \
--no-amp --no-open-gop --pools + --keyint 240 --min-keyint 1 --bframes 8 --max-merge 4 --ref 4 --weightb --rc-lookahead 72 \
--scenecut 40 --no-strong-intra-smoothing --vbv-bufsize 160000 --vbv-maxrate 160000 --input-depth 10 --output 265.mkv -"

remote_stmt=f'zstdcat ~/yrm/src| tee \
>({x264_stmt} 2>&1 | tee x264.log) \
>({x265_stmt} 2>&1 | tee x265.log) \
>/dev/null{newline}'

with open("x26x.sh","w") as f:
    f.write(remote_stmt)
remote_x265stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0 -T8|ssh {remotehost} "cat>~/y4m/src"'
remote_convertcmd=f'scp {os.getcwd()}\\x26x.sh {remotehost}:{remotecwd}'

print("==========\n远程执行编码 x265:")
cmd=f'set MOVIE={MOVIE}{newline}'
cmd+=f'set TEST=NO{newline}'
cmd+=remote_x265stmt+newline
cmd+=remote_convertcmd+newline
cmd+=f'pause{newline}'
cmd+=f'scp {remotehost}:{remotecwd}/264.mkv {os. getcwd()}\\264.mkv{newline}'
cmd+=f'scp {remotehost}:{remotecwd}/265.mkv {os. getcwd()}\\265.mkv{newline}'

pyperclip.copy(cmd)
print(cmd)

远程执行编码 x265:
set MOVIE=C:\Users\heber\Music\How.to.Train.Your.Dragon.2.2014.2160p.BluRay.REMUX.HEVC.DTS-MA.7.1-PTHOME\How.to.Train.Your.Dragon.2.2014.2160p.BluRay.REMUX.HEVC.DTS-MA.7.1-PTHOME.mkv
set TEST=NO
vspipe.exe --y4m "C:\Users\heber\Documents\Compressor\vs.py" - |zstd -0 -T8|ssh ubuntu20local "cat>~/y4m/src"
scp C:\Users\heber\Documents\Compressor\x26x.sh ubuntu20local:~/compressor
pause
scp ubuntu20local:~/compressor/265.mkv C:\Users\heber\Documents\Compressor\265.mkv



# Step4: Remux MKV
## x264

In [17]:
# title="".join(re.findall(r'([^\\]*\d{4}p)', MOVIE))
# NEWMOVIE=f'{os.getcwd()}\\{title}.BluRay.x264.DTS-PTH.mkv'
x264_filename="".join(re.findall(r'([^\\]*$)', MOVIE)).replace("REMUX.AVC", "x264")
x264_foldername=PROCUCT_DEST_DIR+"\\"+x264_filename.replace(".mkv",'')
NEWMOVIE=f'{x264_foldername}\\{x264_filename}'
print(x264_filename)
cmd=f'mkdir {x264_foldername} ^& '
cmd+=f'{os. getcwd()}\\mkvtoolnix\\mkvmerge.exe -o "{NEWMOVIE}" -D ( "{MOVIE}" )\
    -A -S -T -M -B --no-chapters ( "{os. getcwd()}\\264.mkv" )'
pyperclip.copy(cmd)
print(f"即将执行: \n   [剩] {MOVIE}\n + [视] {os. getcwd()}\\264.mkv\n-> [全] {NEWMOVIE}\n=================")
print(cmd)

Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome.mkv
即将执行: 
   [剩] C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.Atmos.TrueHD.7.1-louco@PTHome.mkv
 + [视] C:\Users\heber\Documents\Compressor\264.mkv
-> [全] C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome.mkv
mkdir C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome ^& C:\Users\heber\Documents\Compressor\mkvtoolnix\mkvmerge.exe -o "C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome.mkv" -D ( "C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.Atmos.TrueHD.7.1-louco@PTHome.mkv" )    -A -S -T -M -B --no-chapters ( "C:\Users\heber\Documents\Compressor\264.mkv" )


In [18]:
subprocess.run(f"start cmd /c {cmd} ^& pause", shell=True)
None

In [22]:
generate_videoinfo(f'{x264_foldername}\\{x264_filename}',"x264.log")
None

[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome
RELEASE DATE..............: 2021-01-12
RELEASE SiZE..............: 17.591 GB
SOURCE....................: 

.Media.Info
RUNTiME...................: 2 h 22 min
ViDEO CODEC...............: x264 High@L4.1 @ 13.35Mb/s
RESOLUTiON................: 1920x800
DiSPLAY ASPECT RATiO......: 2.400
FRAME RATE................: 23.976
BIT DEPTH.................: 8 bits
AUDiO CODEC...............: English MLP FBA 8 @ 4271.44Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : Chinese(chs)
SUBTiTLES.................: S_HDMV/PGS : English(en)
SUBTiTLES.................: S_HDMV/PGS : Chinese(chs&eng)


.x264.Info
x264 [info]: profile High, level 4.1, 4:2:0, 8-bit
x264 [info]: frame I:1182  Avg QP:17.00  size:270311
x264 [info]: frame P:49116 Avg QP:18.81  size:113585
x264 [info]: frame B:154121 Avg QP:20.83  size: 54055
x264 [info]: 

In [51]:
get_captures("x264", x264_filename)

Capture x264


## x265

In [24]:
x265_filename="".join(re.findall(r'([^\\]*$)', MOVIE)).replace("REMUX.AVC", "x265.10bit")
x265_foldername=PROCUCT_DEST_DIR+"\\"+x265_filename.replace(".mkv",'')
NEWMOVIE=f'{x265_foldername}\\{x265_filename}'
cmd=f'mkdir {x264_foldername} ^& '
cmd+=f'{os. getcwd()}\\mkvtoolnix\\mkvmerge.exe -o "{NEWMOVIE}" -D ( "{MOVIE}" )\
    -A -S -T -M -B --no-chapters ( "{os. getcwd()}\\265.mkv" )'
pyperclip.copy(cmd)
print(f"即将执行: \n   [剩] {MOVIE}\n + [视] {os. getcwd()}\\265.mkv\n-> [全] {NEWMOVIE}\n=================")
print(cmd)

即将执行: 
   [剩] C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.Atmos.TrueHD.7.1-louco@PTHome.mkv
 + [视] C:\Users\heber\Documents\Compressor\265.mkv
-> [全] C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x265.10bit.Atmos.TrueHD.7.1-louco@PTHome\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x265.10bit.Atmos.TrueHD.7.1-louco@PTHome.mkv
mkdir C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome ^& C:\Users\heber\Documents\Compressor\mkvtoolnix\mkvmerge.exe -o "C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x265.10bit.Atmos.TrueHD.7.1-louco@PTHome\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x265.10bit.Atmos.TrueHD.7.1-louco@PTHome.mkv" -D ( "C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.Atmos.TrueHD.7.1-louco@PTHome.mkv" )    -A -S -T -M -B --no-chapters ( "C:\Users\heber\Documents\Compressor\265.mkv" )


In [25]:
subprocess.run(f"start cmd /c {cmd} ^& pause", shell=True)

CompletedProcess(args='start cmd /c mkdir C:\\Users\\heber\\Music\\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome ^& C:\\Users\\heber\\Documents\\Compressor\\mkvtoolnix\\mkvmerge.exe -o "C:\\Users\\heber\\Music\\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x265.10bit.Atmos.TrueHD.7.1-louco@PTHome\\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x265.10bit.Atmos.TrueHD.7.1-louco@PTHome.mkv" -D ( "C:\\Users\\heber\\Music\\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.Atmos.TrueHD.7.1-louco@PTHome.mkv" )    -A -S -T -M -B --no-chapters ( "C:\\Users\\heber\\Documents\\Compressor\\265.mkv" ) ^& pause', returncode=0)

In [32]:
generate_videoinfo(x265_filename,"x265.log")
None

[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: A Gentleman.2017.1080p.BluRay.x265.10bit-louco@PTH
RELEASE DATE..............: 2020-12-15
RELEASE SiZE..............: 10.446 GB
SOURCE....................: A Gentleman.2017.1080p.BluRay.REMUX.AVC-louco@PTH

.Media.Info
RUNTiME...................: 2 h 12 min
ViDEO CODEC...............: x265 Main 10@L5.1@High @ 9.31Mb/s
RESOLUTiON................: 1920x800
DiSPLAY ASPECT RATiO......: 2.400
FRAME RATE................: 24.000
BIT DEPTH.................: 10 bits
AUDiO CODEC...............: Hindi DTS 5.1 @ 1473.63Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : English(en)
SUBTiTLES.................: S_TEXT/ASS : Chinese(cht&eng)
SUBTiTLES.................: S_TEXT/ASS : Chinese(cht)
SUBTiTLES.................: S_TEXT/ASS : Chinese(chs&eng)
SUBTiTLES.................: S_TEXT/ASS : Chinese(chs)


.x265.Info
[/quote]


In [39]:
print(x265_filename)
get_captures("x265", x265_filename)

A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.x265.10bit.DTS-louco@PTH.mkv
Capture x265


In [34]:
get_mediainfo(x265_filename)
None

Copy Mediainfo


# Step5 发布

In [43]:
def get_bbcode(file, src="",logname=""):
    print("Title:", file.replace("."," "))
#     由ZERO13@PTHome字幕组完成校对、调整。
    picbed='''
[url=https://sm.ms/image/bNrH6hTY4oIEjla][img]https://i.loli.net/2021/01/06/bNrH6hTY4oIEjla.jpg[/img][/url]
[url=https://sm.ms/image/TFwQIK4VOsnUiW9][img]https://i.loli.net/2021/01/06/TFwQIK4VOsnUiW9.jpg[/img][/url]
[url=https://sm.ms/image/KGM6as9Xyp1oxqf][img]https://i.loli.net/2021/01/06/KGM6as9Xyp1oxqf.jpg[/img][/url]
[url=https://sm.ms/image/nlVLM3adumEHScA][img]https://i.loli.net/2021/01/06/nlVLM3adumEHScA.jpg[/img][/url]
[url=https://sm.ms/image/9dQrFLlCh51csRP][img]https://i.loli.net/2021/01/06/9dQrFLlCh51csRP.jpg[/img][/url]
[url=https://sm.ms/image/TjNZKcG3USAO8EQ][img]https://i.loli.net/2021/01/06/TjNZKcG3USAO8EQ.jpg[/img][/url]
    '''
    if "REMUX" in file:
        picbed='''
[url=https://sm.ms/image/FjRpzKqL2rkdGCA][img]https://i.loli.net/2021/01/06/FjRpzKqL2rkdGCA.jpg[/img][/url]
[url=https://sm.ms/image/DR79FMJIpYS1O42][img]https://i.loli.net/2021/01/06/DR79FMJIpYS1O42.jpg[/img][/url]
[url=https://sm.ms/image/nHLCScypZ4wVP7N][img]https://i.loli.net/2021/01/06/nHLCScypZ4wVP7N.jpg[/img][/url]
[url=https://sm.ms/image/LilaOG3cDn7qzQg][img]https://i.loli.net/2021/01/06/LilaOG3cDn7qzQg.jpg[/img][/url]
[url=https://sm.ms/image/ZYVtgW8Eo45McDG][img]https://i.loli.net/2021/01/06/ZYVtgW8Eo45McDG.jpg[/img][/url]
[url=https://sm.ms/image/sHNcvD1mP8bCkoB][img]https://i.loli.net/2021/01/06/sHNcvD1mP8bCkoB.jpg[/img][/url]
    '''
    bbcode='''
[quote][size=5][color=RoyalBlue]原盘来自TTG，字幕素材来自人人字幕组
字幕转载务必注意礼节，请保留原作者信息，谢绝二次提取修改！
在此感谢各位原作者及分享者！如有侵权请联系删除！[/color][/size][/quote]
[img]https://pic.imgdb.cn/item/5ff590053ffa7d37b3bd4495.jpg[/img]

◎译　　名　怀旧范儿的狂欢节
◎片　　名　A Good Old Fashioned Orgy
◎年　　代　2011
◎产　　地　美国
◎类　　别　喜剧
◎语　　言　英语
◎上映日期　2011-09-02(美国)
◎IMDb评分  6.1/10 from 21101 users
◎IMDb链接  https://www.imdb.com/title/tt1231586
◎豆瓣评分　5.6/10 from 1038 users
◎豆瓣链接　https://movie.douban.com/subject/3158436/
◎片　　长　95分钟
◎导　　演　亚历克斯·格瑞格里 Alex Gregory / 皮特·海克 Peter Huyck
◎编　　剧　亚历克斯·格瑞格里 Alex Gregory / 皮特·海克 Peter Huyck
◎主　　演　莱丝莉·比伯 Leslie Bibb
　　　　　  林赛·斯隆 Lindsay Sloane
　　　　　  蕾克·贝尔 Lake Bell
　　　　　  林·沙烨 Lin Shaye
　　　　　  威尔·福特 Will Forte
　　　　　  杰森·苏戴奇斯 Jason Sudeikis
　　　　　  米歇尔·博斯 Michelle Borth
　　　　　  尼克·克罗尔 Nick Kroll
　　　　　  William Hutchings
　　　　　  Luka Apt
　　　　　  Aaron Pushkar
　　　　　  Lauren Crawford
　　　　　  露茜·彭奇 Lucy Punch
　　　　　  Jason Ciok
　　　　　  Kristin Mellian
　　　　　  乔·蒙坦蒂 Joe Montanti
　　　　　  马丁·斯塔尔 Martin Starr
　　　　　  瑞斯·考罗 Rhys Coiro
　　　　　  泰勒·莱伯恩 Tyler Labine
　　　　　  大卫·科恩查内 David Koechner
　　　　　  唐·约翰逊 Don Johnson
　　　　　  Jack Landry
　　　　　  安吉拉·萨拉弗安 Angela Sarafyan


◎标　　签　喜剧 | 美国 | 2011 | 情色 | 文艺 | 剧情 | 友谊,性,自信,偶尔的放纵,缓解压力,羡慕!!! | Sex

◎简　　介  

　　在富豪老爹的豪宅里，三十岁的埃里克（杰森·苏戴奇斯 Jason Sudeikis 饰）过着衣食无忧的生活，趁着老爹不在家，他还会召集狐朋狗友们举办疯狂的派对，派对俨然成为了埃里克生命中不可分割的一部分。突然有一天，这样惬意的生活走到了尽头，老爹告诉埃里克，他要把豪宅卖掉。为了纪念曾经的生活，埃里克决定在豪宅中举办最后一次派对，这一消息让埃里克的朋友们个个蠢蠢欲动，每一个人都将生命中的种种希望寄托于这场空前盛大的狂欢节中。与此同时，为了能够尽可能长时间的将房子留在手中，埃里克开始和负责此宗买卖的房产经纪人凯丽（莱丝莉·比伯 Leslie Bibb 饰）约会，令埃里可没有想到的是，自己竟然陷入了凯丽的温柔乡中无法自拔。

[img]https://images2.imgbox.com/04/6b/Ggp5ReQb_o.png[/img]

'''+f'{generate_videoinfo(file,logname,src=src)}\n{picbed}\n{get_mediainfo(file)}'
    pyperclip.copy(bbcode)

In [48]:
from utils import *
REMUX=r"C:\Users\heber\Music\A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.REMUX.AVC.DTS-HD 5.1-louco@PTH\A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.REMUX.AVC.DTS-HD 5.1-louco@PTH.mkv"
SRC=r"A Good Old Fashioned Orgy 2011 Blu-ray RUS 1080p AVC DTS-HD 5.1-hdsng@HDSky"
get_bbcode(REMUX,src=SRC)

Title: C:\Users\heber\Music\A Good Old Fashioned Orgy 2011 Blu-ray RUS 1080p REMUX AVC DTS-HD 5 1-louco@PTH\A Good Old Fashioned Orgy 2011 Blu-ray RUS 1080p REMUX AVC DTS-HD 5 1-louco@PTH mkv
[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.REMUX.AVC.DTS-HD 5.1-louco@PTH
RELEASE DATE..............: 2021-01-06
RELEASE SiZE..............: 18.259 GB
SOURCE....................: A Good Old Fashioned Orgy 2011 Blu-ray RUS 1080p AVC DTS-HD 5.1-hdsng@HDSky

.Media.Info
RUNTiME...................: 1 h 34 min
ViDEO CODEC...............: ? High@L4.1 @ 24.0Mb/s
RESOLUTiON................: 1920x1080
DiSPLAY ASPECT RATiO......: 1.778
FRAME RATE................: 24.000
BIT DEPTH.................: 8 bits
AUDiO CODEC...............: Russian DTS-HD MA 5.1 @ 3702.48Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : Chinese(zhs)
SUBTiTLES.................: S_HDMV/PGS : Chinese(zht)
SUBTiTLES...............

In [47]:
X264FINAL=r"C:\Users\heber\Music\A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.x264.DTS-louco@PTH\A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.x264.DTS-louco@PTH.mkv"
get_bbcode(X264FINAL,src=SRC,logname="x264.log")

Title: C:\Users\heber\Music\A Good Old Fashioned Orgy 2011 Blu-ray RUS 1080p x264 DTS-louco@PTH\A Good Old Fashioned Orgy 2011 Blu-ray RUS 1080p x264 DTS-louco@PTH mkv
[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.x264.DTS-louco@PTH
RELEASE DATE..............: 2021-01-06
RELEASE SiZE..............: 11.457 GB
SOURCE....................: A Good Old Fashioned Orgy 2011 Blu-ray RUS 1080p AVC DTS-HD 5.1-hdsng@HDSky

.Media.Info
RUNTiME...................: 1 h 34 min
ViDEO CODEC...............: x264 High@L4.1 @ 16.47Mb/s
RESOLUTiON................: 1920x1036
DiSPLAY ASPECT RATiO......: 1.853
FRAME RATE................: 24.000
BIT DEPTH.................: 8 bits
AUDiO CODEC...............: Russian DTS 5.1 @ 768.0Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : Chinese(zhs)
SUBTiTLES.................: S_HDMV/PGS : Chinese(zht)
SUBTiTLES.................: S_HDMV/PGS : Russian(ru)


.x264.Inf

In [49]:
X265FINAL=r"C:\Users\heber\Music\A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.x265.10bit.DTS-louco@PTH\A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.x265.10bit.DTS-louco@PTH.mkv"
# get_captures("x265",X265FINAL)
get_bbcode(X265FINAL,src=SRC,logname="x265.log")

Title: C:\Users\heber\Music\A Good Old Fashioned Orgy 2011 Blu-ray RUS 1080p x265 10bit DTS-louco@PTH\A Good Old Fashioned Orgy 2011 Blu-ray RUS 1080p x265 10bit DTS-louco@PTH mkv
[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.x265.10bit.DTS-louco@PTH
RELEASE DATE..............: 2021-01-06
RELEASE SiZE..............: 10.563 GB
SOURCE....................: A Good Old Fashioned Orgy 2011 Blu-ray RUS 1080p AVC DTS-HD 5.1-hdsng@HDSky

.Media.Info
RUNTiME...................: 1 h 34 min
ViDEO CODEC...............: x265 Main 10@L5.1@High @ 15.12Mb/s
RESOLUTiON................: 1920x1036
DiSPLAY ASPECT RATiO......: 1.853
FRAME RATE................: 24.000
BIT DEPTH.................: 10 bits
AUDiO CODEC...............: Russian DTS 5.1 @ 768.0Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : Chinese(zhs)
SUBTiTLES.................: S_HDMV/PGS : Chinese(zht)
SUBTiTLES.................: S_HDMV/PG

# 备忘录